# Crime Trail Blazers

In [22]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [23]:
# Name of the CSV file
file15 = 'Resources/2015_Portland_Crime_Data.csv'
file16 = 'Resources/2016_Portland_Crime_Data.csv'
file17 = 'Resources/2017_Portland_Crime_Data.csv'
file18 = 'Resources/2018_Portland_Crime_Data.csv'
file19 = 'Resources/2019YTD_Portland_Crime_Data.csv'
nba_data ='Resources/portland.games.stats.csv'
#port_home = "Resources/port_home.csv"

In [24]:
# The correct encoding must be used to read the CSV in pandas
df_2015 = pd.read_csv(file15, encoding="ISO-8859-1")
df_2016 = pd.read_csv(file16, encoding="ISO-8859-1")
df_2017 = pd.read_csv(file17, encoding="ISO-8859-1")
df_2018 = pd.read_csv(file18, encoding="ISO-8859-1")
df_2019 = pd.read_csv(file18, encoding="ISO-8859-1")
nba_data = pd.read_csv(nba_data, encoding="ISO-8859-1")

## Portland Trailblazers Stats 2015-18

In [25]:
nba_data.head()

,Unnamed: 0,Team,Game,Date,Home,Opponent,WINorLOSS,TeamPoints,OpponentPoints,FieldGoals,...,Opp.FreeThrows,Opp.FreeThrowsAttempted,Opp.FreeThrows.,Opp.OffRebounds,Opp.TotalRebounds,Opp.Assists,Opp.Steals,Opp.Blocks,Opp.Turnovers,Opp.TotalFouls
0,1108,POR,1,10/29/2014,Home,OKC,W,106,89,39,...,21,26,0.808,14,43,19,9,5,13,28
1,2108,POR,2,10/31/2014,Away,SAC,L,94,103,34,...,31,35,0.886,14,49,14,6,2,10,24
2,3108,POR,3,11/2/2014,Home,GSW,L,90,95,37,...,17,21,0.810,7,41,22,11,7,17,16
3,4108,POR,4,11/4/2014,Home,CLE,W,101,82,37,...,11,14,0.786,13,41,18,5,3,14,19
4,5108,POR,5,11/6/2014,Home,DAL,W,108,87,40,...,16,16,1.000,8,34,14,6,4,7,17


In [26]:
# add Day of Week
nba_data['dow']= pd.to_datetime(nba_data['Date']).dt.dayofweek

In [27]:
# Change date format and reduce dates to 2015 - 2018
nba_data['Date'] = pd.to_datetime(nba_data['Date'])
start_date = '01/01/2015'
end_date = '12/31/2018'
mask = (nba_data['Date'] >= start_date) & (nba_data['Date'] <= end_date)
nba_data = nba_data.loc[mask]

In [28]:
# Created a new dataframe using the data_df to display these columns:
new_nba_df = nba_data[['Date','Team','Game','Home','WINorLOSS', 'dow']] 
new_nba_df.head()

,Date,Team,Game,Home,WINorLOSS,dow
33,2015-01-03,POR,34,Home,L,5
34,2015-01-05,POR,35,Home,W,0
35,2015-01-08,POR,36,Home,W,3
36,2015-01-10,POR,37,Home,W,5
37,2015-01-11,POR,38,Away,W,6


In [41]:
home_blazers_df = new_nba_df.loc[new_nba_df["Home"] == "Home"].reset_index(drop=True)
home_blazers_df.head()

,Date,Team,Game,Home,WINorLOSS,dow
0,2015-01-03,POR,34,Home,L,5
1,2015-01-05,POR,35,Home,W,0
2,2015-01-08,POR,36,Home,W,3
3,2015-01-10,POR,37,Home,W,5
4,2015-01-14,POR,39,Home,L,2


## Crime Data

In [30]:
# Concatinate all crime data
concat_crime_df = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019])
concat_crime_df.head()

,Address,CaseNumber,CrimeAgainst,Neighborhood,OccurDate,OccurTime,OffenseCategory,OffenseType,OpenDataLat,OpenDataLon,OpenDataX,OpenDataY,ReportDate,OffenseCount
0,NaN,15-X197430,Person,Piedmont,5/12/2015,1400,Assault Offenses,Intimidation,NaN,NaN,NaN,NaN,5/12/2015,1
1,NaN,15-X4282999,Person,Buckman West,5/1/2015,2143,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,5/1/2015,1
2,NaN,15-X4283033,Person,University Park,5/1/2015,1625,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,5/1/2015,1
3,NaN,15-X4283218,Person,Madison South,5/1/2015,1820,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,5/1/2015,1
4,NaN,15-X4283218,Person,Madison South,5/1/2015,1820,Kidnapping/Abduction,Kidnapping/Abduction,NaN,NaN,NaN,NaN,5/1/2015,1


In [43]:
# add Day of Week
concat_crime_df['DOW']= pd.to_datetime(concat_crime_df['OccurDate']).dt.dayofweek

In [45]:
# Created a new dataframe using the data_df to display these columns:
new_crime_df = concat_crime_df[['OccurDate','Neighborhood','OffenseType','DOW']]

In [46]:
# Rename date field
new_crime_df = new_crime_df.rename(columns={"OccurDate":"Date"})
new_crime_df.head()

,Date,Neighborhood,OffenseType,DOW
0,5/12/2015,Piedmont,Intimidation,1
1,5/1/2015,Buckman West,Simple Assault,4
2,5/1/2015,University Park,Simple Assault,4
3,5/1/2015,Madison South,Simple Assault,4
4,5/1/2015,Madison South,Kidnapping/Abduction,4


In [47]:
# Filter out reports of crimes prior to 2015
new_crime_df['Date'] = pd.to_datetime(new_crime_df['Date'])
start_date = '01/01/2015'
end_date = '12/31/2018'
mask = (new_crime_df['Date'] >= start_date) & (new_crime_df['Date'] <= end_date)
new_crime_df = new_crime_df.loc[mask]
new_crime_df

,Date,Neighborhood,OffenseType,DOW
0,2015-05-12,Piedmont,Intimidation,1
1,2015-05-01,Buckman West,Simple Assault,4
2,2015-05-01,University Park,Simple Assault,4
3,2015-05-01,Madison South,Simple Assault,4
4,2015-05-01,Madison South,Kidnapping/Abduction,4
...,...,...,...,...
60383,2018-05-07,NaN,Drug/Narcotic Violations,0
60384,2018-07-25,NaN,Simple Assault,2
60385,2018-10-02,Arlington Heights,Vandalism,1
60386,2018-02-27,NaN,All Other Larceny,1


In [54]:
# Create new dataframe specific to Date and Days of week
df_dow = new_crime_df[["Date", "DOW"]]

In [50]:
# Run for loop to count number of crimes for each date
unique_dates = list(df_dow["Date"].unique())

listy = []
for i in unique_dates:
    holdy = df_dow.loc[df_dow["Date"] == i].reset_index(drop=True)
    holdy1 = holdy["Date"].count()
    holder_dictionary = {"Date": i,
                         "Num_times": holdy1,
                         "DOW": holdy["DOW"][0]}
    listy.append(holder_dictionary)

In [53]:
# Create datafrom of the daily crime count
crime_count = pd.DataFrame(listy)
crime_count

,Date,Num_times,DOW
0,2015-05-12,161,1
1,2015-05-01,196,4
2,2015-05-02,176,5
3,2015-05-03,130,6
4,2015-05-04,143,0
...,...,...,...
1443,2018-12-28,276,4
1444,2018-12-29,226,5
1445,2018-12-30,242,6
1446,2018-12-31,176,0


## Trail Blazers and Crime Data